# Objectives
YWBAT
* use the spotify API to make calls 
* build a spotify api class in Python
* use a decorator to refresh an api token 

# Outline
* read in spotify credentials
* make a request for an access token
* build that out into functions
* write spotify api class
* write method to get an access token 
* write decorator for freshing token

In [14]:
import os
import requests
import base64
import datetime
from functools import wraps

In [15]:
with open("/Users/rcarrasco/.ssh/spotify.json") as f:
    d = json.load(f)
    client_id = d['client_id']
    client_secret = d['client_secret']

In [85]:
class SpotifyAPI():
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
        self.access_token = None
        pass
    
    
    class Decorators():
        @staticmethod
        def refreshToken(decorated):
            @wraps(decorated)
            def wrapper(api, *args, **kwargs):
                now = datetime.datetime.now()
                if now > api.expires_at:
                    api.get_access_token()
                return decorated(api, *args, **kwargs)
            return wrapper
    
    
    def get_encoded_client_credentials(self):
        client_credentials = f"{self.client_id}:{self.client_secret}"
        client_credentials_b64 = base64.b64encode(client_credentials.encode())
        return client_credentials_b64 


    def get_access_token(self):
        token_url = "https://accounts.spotify.com/api/token"
        method="POST"
        data = {"grant_type": "client_credentials"}
        client_credentials_b64 = get_encoded_client_credentials(self.client_id, self.client_secret)
        headers = {"Authorization": f"Basic {client_credentials_b64.decode()}"}
        r = requests.request(method=method, url=token_url, data=data, headers=headers)
        self.access_token = r.json()['access_token']
        now = datetime.datetime.now()
        expires_in = r.json()['expires_in']
        self.expires_at = now + datetime.timedelta(seconds=expires_in)
        return  
    
    
    def get_bearer_token_headers(self):
        if self.access_token is None:
            raise Exception("No Access Token Found")
        return {"Authorization": f"Bearer {self.access_token}"}
            
        
    @Decorators.refreshToken
    def search(self, **params):
        """
        **params takes in spotify arguments
        q: (required) query term
        type: (required) type of spotify object album, playlist, artist, etc
        market: (optional) market of country you want to query
        
        example:
        search(q='the dark knight', type='album')
        
        """
        url = "https://api.spotify.com/v1/search"
        method = "GET"
        bearer_token_headers = self.get_bearer_token_headers()
        r = requests.request(method=method, url=url, headers=bearer_token_headers, params=params)
        return r 
    
    
    @Decorators.refreshToken
    def search_tracks(self, endpoint_path='tracks', track_id=None, **params):
        url = "https://api.spotify.com/v1"
        url = os.path.join(url, endpoint_path)
        if track_id:
            url = os.path.join(url, track_id)
        print(url)
    


In [86]:
client = SpotifyAPI(client_id=client_id, client_secret=client_secret)

In [87]:
client.search_tracks()

https://api.spotify.com/v1/tracks


# What did we learn? 
* base64 and decorators
* spotify has a lot of endpoints and options
* I did not know you could make a static method and call if before the function with a decorator
* making useful function for search queries and general api things
* wraps from functools